### Constraint 1

#### Library

In [ ]:
pip install wikipedia-api nltk

In [ ]:
import wikipediaapi
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, spearmanr
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Scrape Data

In [ ]:
user_agent = "MyWikipediaScraper/1.0 (https://mywebsite.com)"
wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})

def get_wikipedia_article(article_name):
    page = wiki_wiki.page(article_name)
    if page.exists():
        return page.text
    return ""

articles = ["Natural language processing", "Machine learning", "Artificial intelligence",
            "Data science", "Computer science", "Mathematics", "Physics", "Chemistry",
            "Biology", "Economics"]

#### Tokenisation

In [ ]:
corpus = ""
for article in articles:
    corpus += get_wikipedia_article(article)

tokens = word_tokenize(corpus.lower())
tokens = tokens[:1000000]  # Limit to the first 1,000,000 tokens
print(f"Total tokens collected: {len(tokens)}")

unique_tokens = set(tokens)
print(f"Total unique tokens: {len(unique_tokens)}")
print(f"Unique tokens: {unique_tokens}")

Total tokens collected: 80365
Total unique tokens: 9024
Unique tokens: {'interactions', 'food', 'catchphrase', '3.5', 'components', 'connectionist', 'contain', 'chern', 'bred', 'algorithmically', 'equilibrium', "n't", 'theory', 'linguistic', 'board', '-', 'hopes', 'frequently', 'useful', 'becoming', 'curlie', 'knime', 'microsystems', '80', 'managing', 'waymo', 'ring-like', '3000', 'companies', '\\delta', 'rent-seeking', 'aversion', 'cosmological', 'hope', '1.', 'eliezer', 'notable', 'murderous', 'chromatids', 'indeed', 'higgs', 'nitrogen', 'indus', 'robbins', 'was', 'agents', 'group', 'cost–benefit', 'measurement', 'exists', 'starting', 'scholarship', 'kitab', 'mercy', 'breakthrough', 'neuromorphic', 'covers', 'butterfly', 'phrasebook', 'promote', 'ir', 'comprise', '9th', 'self-interested', 'eleven', 'publications', '1997', 'guarantees', '60+/100+', 'visibility', 'vehicles', 'optimizing', 'expect', 'units', 'sustaining', 'debris', 'commonsensical', 'observing', 'μαθηματικοί', 'countere

#### Functions

In [ ]:
def build_co_occurrence_matrix(corpus, vocab_size, window_size):
    co_occurrence_matrix = np.zeros((vocab_size, vocab_size), dtype=np.float32)
    word_to_index = {word: idx for idx, word in enumerate(set(corpus))}
    for i, word in enumerate(corpus):
        word_idx = word_to_index[word]
        start = max(0, i - window_size)
        end = min(len(corpus), i + window_size + 1)
        context = [corpus[j] for j in range(start, end) if j != i]
        for context_word in context:
            context_word_idx = word_to_index[context_word]
            co_occurrence_matrix[word_idx, context_word_idx] += 1
    return co_occurrence_matrix, word_to_index

def calculate_similarity(word1, word2, word_embeddings, word_to_index):
    if word1 in word_to_index and word2 in word_to_index:
        vec1 = word_embeddings[word_to_index[word1]].reshape(1, -1)
        vec2 = word_embeddings[word_to_index[word2]].reshape(1, -1)
        return cosine_similarity(vec1, vec2)[0][0]
    else:
        return None

#### Result and Training

In [ ]:
df = pd.read_csv('/content/SimLex-999.txt', delimiter='\t')

results = {}

window_sizes = [2, 5, 10]

for window_size in window_sizes:
    co_occurrence_matrix, word_to_index = build_co_occurrence_matrix(tokens, len(set(tokens)), window_size)

    total_count = np.sum(co_occurrence_matrix)
    sum_over_words = np.sum(co_occurrence_matrix, axis=0)
    expected_counts = np.outer(sum_over_words, sum_over_words) / total_count
    ppmi_matrix = np.maximum(np.log(co_occurrence_matrix / expected_counts), 0)

    svd = TruncatedSVD(n_components=100)
    word_embeddings = svd.fit_transform(ppmi_matrix)

    df['predicted_similarity'] = df.apply(lambda row: calculate_similarity(row['word1'], row['word2'], word_embeddings, word_to_index), axis=1)
    df_filtered = df.dropna(subset=['predicted_similarity'])
    pearson_corr = pearsonr(df_filtered['SimLex999'], df_filtered['predicted_similarity'])[0]
    spearman_corr = spearmanr(df_filtered['SimLex999'], df_filtered['predicted_similarity'])[0]
    results[window_size] = {
        'pearson': pearson_corr,
        'spearman': spearman_corr
    }

for window_size, result in results.items():
    print(f'Window Size: {window_size}')
    print(f'Pearson Correlation: {result["pearson"]}')
    print(f'Spearman Correlation: {result["spearman"]}')
    print('---')


<ipython-input-10-97998d84d2de>:13: RuntimeWarning: divide by zero encountered in log
  ppmi_matrix = np.maximum(np.log(co_occurrence_matrix / expected_counts), 0)
<ipython-input-10-97998d84d2de>:13: RuntimeWarning: divide by zero encountered in log
  ppmi_matrix = np.maximum(np.log(co_occurrence_matrix / expected_counts), 0)
<ipython-input-10-97998d84d2de>:13: RuntimeWarning: divide by zero encountered in log
  ppmi_matrix = np.maximum(np.log(co_occurrence_matrix / expected_counts), 0)


Window Size: 2
Pearson Correlation: 0.06962361055102537
Spearman Correlation: 0.09138243472587387
---
Window Size: 5
Pearson Correlation: 0.08966630813361028
Spearman Correlation: 0.09368264218944367
---
Window Size: 10
Pearson Correlation: 0.05128387522219853
Spearman Correlation: 0.031106934819627688
---


### Constraint 2

In [1]:
pip install transformers datasets torch scipy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [2]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr

#### Load Dataset

In [ ]:
dataset = load_dataset('wikipedia', '20220301.en', split='train[:1%]')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/64587 [00:00<?, ? examples/s]

#### Model

In [ ]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()

#### Word Embeddings

In [ ]:
def get_finetuned_word_embedding(word):
    inputs = tokenizer(word, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

#### Process the Test Data

In [ ]:
df = pd.read_csv('SimLex-999.txt', delimiter='\t')
similarity_scores = []
for _, row in df.iterrows():
    word1, word2 = row['word1'], row['word2']
    try:
        vec1 = get_finetuned_word_embedding(word1)
        vec2 = get_finetuned_word_embedding(word2)
        similarity = cosine_similarity([vec1], [vec2])[0][0]
    except KeyError:
        similarity = None
    similarity_scores.append(similarity)

df['predicted_similarity'] = similarity_scores
df_filtered = df.dropna(subset=['predicted_similarity'])
spearman_corr = spearmanr(df_filtered['SimLex999'], df_filtered['predicted_similarity'])[0]
print(f'Spearman Rank Correlation: {spearman_corr:.4f}')